In [14]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

In [15]:
# 市場データの取得
from pandas_datareader import data
marketData=['NIKKEI225','SP500','NASDAQCOM']
df_market = data.DataReader(marketData,'fred','2014-01-01', '2023-1-31').asfreq("D")
df_market = df_market.fillna(method='ffill')
df_market = df_market.dropna(how='any')


In [16]:
# 各指数の移動平均との乖離率を算出する
for index_name in df_market.columns:
    for i in [5,10,30,60,90,120]:
        df_market[index_name + "_" + str(i) + "days_diffrol"] =\
        (df_market[index_name].rolling(i).mean() - df_market[index_name]) / df_market[index_name].rolling(i).mean()

In [17]:
# 生データは削除する
df_market = df_market.drop(columns=index_name)

In [18]:
# 予測する企業の株価データを取得
from pandas_datareader.stooq import StooqDailyReader
CODE = 4755
CODE_str = str(CODE) + ".JP"
start = datetime(2014, 1, 1)
end = datetime(2023, 1, 31)

df_target = StooqDailyReader(CODE_str, start=start, end=end).read()
df_target = df_target.sort_values('Date')
df_target = pd.DataFrame(df_target).asfreq("D", method="ffill")


In [19]:
# 目的変数の作成（30日後に5 %以上上昇する時に1）
df_target["30days_after"] = df_target["Close"].shift(-30)
df_target['ratio'] = df_target['30days_after'] / df_target["Close"]
df_target['target'] = np.where(df_target['ratio'] > 1.1, 1, 0)

In [20]:
# 不要な行と列を削除
df_target = df_target.dropna(how='any')
df_target = df_target.drop(columns=["ratio","30days_after"])

In [21]:
# データの結合
df = pd.merge(df_target, df_market, how='left', left_index=True, right_index=True)
df = df.dropna(how='any')

In [22]:
# トレインとテストに分割
df_train = df[:"2021"]
df_test = df["2022":]

In [23]:
# 目的変数と説明変数に分割
X_train = df_train.drop("target", axis=1)
y_train = df_train["target"]
X_test = df_test.drop("target", axis=1)
y_test = df_test["target"]

In [24]:
# 学習
from sklearn.linear_model import LogisticRegression

In [26]:
# ロジスティック回帰のインスタンス
model = LogisticRegression(penalty='l2',          # 正則化項(L1正則化 or L2正則化が選択可能)
                           dual=False,            # Dual or primal
                           tol=0.0001,            # 計算を停止するための基準値
                           C=1.0,                 # 正則化の強さ
                           fit_intercept=True,    # バイアス項の計算要否
                           intercept_scaling=1,   # solver=‘liblinear’の際に有効なスケーリング基準値
                           class_weight=None,     # クラスに付与された重み
                           random_state=None,     # 乱数シード
                           solver='lbfgs',        # ハイパーパラメータ探索アルゴリズム
                           max_iter=100,          # 最大イテレーション数
                           multi_class='auto',    # クラスラベルの分類問題（2値問題の場合'auto'を指定）
                           verbose=0,             # liblinearおよびlbfgsがsolverに指定されている場合、冗長性のためにverboseを任意の正の数に設定
                           warm_start=False,      # Trueの場合、モデル学習の初期化に前の呼出情報を利用
                           n_jobs=None,           # 学習時に並列して動かすスレッドの数
                           l1_ratio=None          # L1/L2正則化比率(penaltyでElastic Netを指定した場合のみ)
                          )

In [27]:
model.fit(X_train, y_train) # モデルの学習
# 予測値と実値との比較
df_result = pd.DataFrame()
df_result["Price"] = df_target["Close"]["2022-01-01":]
df_result["true"] = y_test
df_result["Pred"] = model.predict(X_test)

In [28]:
print(df_result)

             Price  true  Pred
Date                          
2022-01-01  1154.0     0     0
2022-01-02  1154.0     0     0
2022-01-03  1154.0     0     0
2022-01-04  1162.0     0     0
2022-01-05  1197.0     0     0
...            ...   ...   ...
2022-12-28   581.0     1     0
2022-12-29   594.0     1     0
2022-12-30   596.0     1     0
2022-12-31   596.0     1     0
2023-01-01   596.0     1     0

[366 rows x 3 columns]


In [29]:
df_result["Pred"]

Date
2022-01-01    0
2022-01-02    0
2022-01-03    0
2022-01-04    0
2022-01-05    0
             ..
2022-12-28    0
2022-12-29    0
2022-12-30    0
2022-12-31    0
2023-01-01    0
Freq: D, Name: Pred, Length: 366, dtype: int32

In [30]:
print("df_result")

df_result


In [31]:
df_result

,Price,true,Pred
Date,,,
2022-01-01,1154.0,0,0
2022-01-02,1154.0,0,0
2022-01-03,1154.0,0,0
2022-01-04,1162.0,0,0
2022-01-05,1197.0,0,0
...,...,...,...
2022-12-28,581.0,1,0
2022-12-29,594.0,1,0
2022-12-30,596.0,1,0


In [32]:
true_count = df_result[df_result["Pred"] == True].shape[0]
false_count = df_result[df_result["Pred"] == False].shape[0]


In [33]:
print("True count: ", true_count)
print("False count: ", false_count)


True count:  0
False count:  366
